#### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import statistics as st

#### Reading Data File

In [2]:
cbcaconh = pd.read_table('cc.pl',header=None, usecols=(1,2,3,4))
cbcaconh.columns=['H','N','C','I']
#cbcaconh[::-100]

#### Asparagine window

In [3]:
ASN=cbcaconh[(cbcaconh.N<120)&(cbcaconh.N>108)]
ASN=ASN[(ASN.H<9.0)&(ASN.H>5.5)]
ASN=ASN[(ASN.C<55)&(ASN.C>34)]
#ASN.reset_index(drop=True)[::-10]

#### Normalization

In [4]:
#%%timeit
ASN=(ASN-ASN.mean())
ASN=ASN.divide(ASN.std(),axis='columns')
#ASN.reset_index(drop=True)[::-10]

#### Finding the best match for each peak by comparing with all other in (N,H) dimensions

In [5]:
#%%timeit 
asn_hn=ASN[['H','N']].values
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(asn_hn)
dist, idx = nbrs.kneighbors(asn_hn) 

#### Distance to nearest neighbor peaks apart

In [6]:
dist_r=dist[:,1:]   # [:,0:] = 0 which represents the distance to it self. which is also treated as neighbor 
dist_r.min()

0.0

#### Matching peaks with the given tolerence distance

In [7]:
tol=0.005
matchs=idx[(np.where(dist_r<tol)[0])] 
matchs.shape

(59, 2)

#### Unique N-H pairs

In [8]:
a=matchs.T[0]
b=matchs.T[1]
unqd=[]
unq=[]
for i in range(len(a)):
    x=a[i]
    y=b[i]
    if (x not in unqd) and (y not in unqd):
        unqd.append(x)
        unqd.append(y)
        unq.append([x,y])
df_hn=(ASN.iloc[unqd])
#df_hn.reset_index(drop=True)

  #### Separating $C_{\alpha}\  and\ \  C_{\beta}$     

In [9]:
tmp=cbcaconh
H=[]
N=[]
CA=[]
CB=[]
for i in range(len(unq)):
    x,y=(ASN.iloc[unq[i]]).index
    Hm=st.mean([tmp.iloc[x,0],tmp.iloc[y,0]])
    Nm=st.mean([tmp.iloc[x,1],tmp.iloc[y,1]])
    cx=tmp.iloc[x,2]
    cy=tmp.iloc[y,2]
    if (cx > 47) and (cy < 47):
        H.append(Hm)
        N.append(Nm)
        CA.append(cx)
        CB.append(cy)
    if (cx < 47) and (cy > 47):
        H.append(Hm)
        N.append(Nm)
        CA.append(cy)
        CB.append(cx)
df_cab=pd.DataFrame({'H':H,'N':N,'CA':CA,'CB':CB})
del(tmp)
#df_cab

#### Normalization

In [10]:
df_hnc=df_cab.copy()
df_hnc=(df_hnc-df_hnc.mean())
df_hnc=df_hnc.divide(df_hnc.std(),axis='columns')
#df_hnc

#### Finding similar peaks in    $N \ - \ C_{\alpha}\ - \ \  C_{\beta}$    

In [11]:
ag_ncc=df_hnc[['N','CA','CB']].values
nbrs_ncc = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(ag_ncc)
dist_ncc, idx_ncc = nbrs_ncc.kneighbors(ag_ncc)
dist_r_ncc=dist_ncc[:,1:]
matchs_ncc=idx_ncc[np.where(dist_r_ncc<0.1)[0]]

#### Unique     $ \ N \ - \ C_{\alpha}\ - \ \  C_{\beta}$   

In [12]:
a=matchs_ncc.T[0]
b=matchs_ncc.T[1]
unqd_ncc=[]
unq_ncc=[]
for i in range(len(a)):
    x=a[i]
    y=b[i]
    if (x not in unqd_ncc) and (y not in unqd_ncc):
        unqd_ncc.append(x)
        unqd_ncc.append(y)
        unq_ncc.append([x,y])
df_ncc=(df_cab.iloc[unqd_ncc])
df_ncc

,CA,CB,H,N
10,52.350,39.648,6.473,109.885
9,52.350,39.648,7.528,109.885
13,52.359,41.735,7.506,113.174
12,52.359,41.735,7.214,113.174
16,52.913,38.182,7.722,115.916
15,52.913,38.182,6.925,115.916
17,54.145,39.209,8.315,113.847
18,54.145,39.209,6.921,113.625
22,54.331,37.484,7.027,112.468
21,54.331,37.484,7.737,112.468
